In [1]:
from collections import defaultdict

import mip
import pandas as pd

In [2]:
dtypes = defaultdict(lambda: pd.Int64Dtype())
dtypes["Workout"] = str
path = "data.csv"
df = pd.read_csv(path, dtype=dtypes)

df = df.fillna(3)
df = df.set_index("Workout")

In [3]:
gender = {"Clara": "f", "Zeini": "f", "Lena": "f", "Rebeca": "f", "Cesar": "m", "Hannes": "m", "Marten": "m", "Johannes": "m"}
female = np.array([gender[n] == "f" for n in df.columns])

In [4]:
matching_problem = mip.Model()

preferences = np.array(df)
names = df.columns
workouts = df.index
x = np.empty_like(preferences, dtype=object)
for i, workout in enumerate(workouts):
    for j, name in enumerate(names):
        x[i, j] = matching_problem.add_var(f"x({workout},{name})", var_type=mip.BINARY)

# Detect two consecutive pauses or workouts
y_both_on = np.empty((len(workouts) - 1, len(names)), dtype=object)
for i, workout in enumerate(workouts[:-1]):
    for j, name in enumerate(names):
        y = matching_problem.add_var(f"y_on({workout},{name})", var_type=mip.BINARY)
        matching_problem += y <= x[i, j]
        matching_problem += y <= x[i + 1, j]
        matching_problem += y >= x[i, j] + x[i + 1, j] - 1
        y_both_on[i, j] = y
y_both_off = np.empty((len(workouts) - 1, len(names)), dtype=object)
for i, workout in enumerate(workouts[:-1]):
    for j, name in enumerate(names):
        y = matching_problem.add_var(f"y_off({workout},{name})", var_type=mip.BINARY)
        matching_problem += y <= 1 - x[i, j]
        matching_problem += y <= 1 - x[i + 1, j]
        matching_problem += y >= 1 - x[i, j] - x[i + 1, j]
        y_both_off[i, j] = y

# Detect if two athletes are doing a workout together
together = np.empty((len(workouts), len(names), len(names)), dtype=object)
for i, workout in enumerate(workouts):
    for j, name_a in enumerate(names):
        for k, name_b in zip(range(j + 1, len(names)), names[j+1:]):
            t = matching_problem.add_var(f"together({i},{j},{k})", var_type=mip.BINARY)
            matching_problem += t <= x[i, j]
            matching_problem += t <= x[i, k]
            matching_problem += t >= x[i, j] + x[i, k] - 1
            together[i, j, k] = t

# Satisfy as many preferences as well as possible
weights = {1: 0, 2: -1, 3: 0, 4: 1, 5: 1}
fixed_weighting = np.vectorize(lambda i: weights[i])(preferences)
# Grant some extra weight to workouts ranked with 5 but scale the bonus by the number of
# total 5s per athlete to prefer wishes from people with fewer 5-rank workouts
all_fives = (preferences == 5)
fives_weighting = (2 * all_fives) / np.clip(all_fives.sum(axis=0), a_min=1, a_max=None)
weighting = fixed_weighting + fives_weighting
matching_problem.objective = mip.maximize((x * weighting).sum())

# Every athlete works out 8 times
for j, name in enumerate(names):
    matching_problem += mip.xsum(x[:, j]) == 8

# Every workout has 1-2 men and 2 women
for i in range(len(workouts)):
    matching_problem += mip.xsum(x[i, female]) == 2
    matching_problem += 1 <= mip.xsum(x[i, ~female]) <= 2

# Nobody works out more than twice in a row
for j in range(len(names)):
    for i in range(len(workouts) - 2):
        matching_problem += mip.xsum(x[i:i+3, j]) <= 2

# Nobody sits out more than twice in a row
for j in range(len(names)):
    for i in range(len(workouts) - 2):
        matching_problem += mip.xsum(x[i:i+3, j]) >= 1

# Nobody is assigned to a workout that they cannot do
matching_problem += mip.xsum(x[preferences == 1]) == 0

# Everybody must do at least one of the first two workouts
for j in range(len(names)):
    matching_problem += mip.xsum(x[:2, j]) >= 1
# Everybody must do at least one of the last two workouts
for j in range(len(names)):
    matching_problem += mip.xsum(x[-2:, j]) >= 1

# Restrict number of consecutive anythings per athlete
for j in range(len(names)):
    matching_problem += mip.xsum(y_both_on[:, j] + y_both_off[:, j]) <= 3

# Ensure that each athlete does some workouts with every other athlete
for j in range(len(names)):
    for k in range(j + 1, len(names)):
        matching_problem += mip.xsum(together[:, j, k]) >= 2

matching_problem.verbose = 0
matching_problem.optimize()

assert matching_problem.num_solutions > 0
print(f"Selecting the 1st of {matching_problem.num_solutions} solutions.")
sol = pd.DataFrame(data=np.vectorize(lambda v: v.x)(x), index=df.index, columns=df.columns, dtype=int)

fives_unsatisfied = (~sol.astype(bool)) & (preferences == 5)
if fives_unsatisfied.to_numpy().any():
    print(f"{np.count_nonzero(fives_unsatisfied)} 5-requests unsatisfied")
    for i, j in zip(*np.nonzero(fives_unsatisfied)):
        print(f"{names[j]} can't do {workouts[i]}")
else:
    print("All 5 requests satisfied")

twos_forced = sol.astype(bool) & (preferences == 2)
if fives_unsatisfied.to_numpy().any():
    print(f"{np.count_nonzero(twos_forced)} forced 2-score workouts")
    for i, j in zip(*np.nonzero(twos_forced)):
        print(f"{names[j]} must do {workouts[i]}")
else:
    print("No forced 2-score workouts")

def highlight_active(x):
    active_color1 = "#4287f5"
    inactive_color1 = "white"
    even = np.arange(x.shape[0]) % 2 == 0
    return np.where((x == 1), f"color: {active_color1}; background-color: {active_color1}", f"color: {inactive_color1}; background-color: {inactive_color1}")
def to_checkmark(value):
    return "✔" if value == 1 else ""

styler = sol.style.apply(highlight_active, axis=1).format(to_checkmark)
styler

Selecting the 1st of 1 solutions.
3 5-requests unsatisfied
Johannes can't do Lupo
Cesar can't do Nate
Cesar can't do Horton
3 forced 2-score workouts
Zeini must do Capoot
Clara must do Wilmot
Rebeca must do Souffront


,Clara,Zeini,Rebeca,Lena,Hannes,Cesar,Marten,Johannes
Workout,,,,,,,,
Lupo,✔,✔,,,,✔,✔,
Nate,,,✔,✔,✔,,,✔
Capoot,,✔,,✔,✔,,✔,
Pheezy,✔,,✔,,,✔,,✔
Adam Brown,,✔,,✔,✔,,✔,
Wilmot,✔,,✔,,,,✔,✔
Wilhelm Tell,,,✔,✔,✔,✔,,
Abbate,✔,✔,,,,,✔,✔
Big Sexy,✔,,,✔,✔,✔,,


In [5]:
styler.to_excel("results.xlsx")